In [1]:
# Start writing code here...

# Introduction
In this project, we analyze audio samples of bird calls, along with metadata labels of location and type of call. We focus on a single species, with the hopes of reducing the wide range of factors producing variability in ecological soundscapes - so we are only exploring intraspecific variations in bird calls. The targeted questions we are hoping to answer are:

- Can we classify gender from bird call audio?
- Can we distinguish birds from different region (same region or different region)?
- Can we classify types of bird calls (e.g. song, call, flight call) from bird call audio?

One motivation for this project is practical and concrete, to make it faster and easier for scientists to collect data on bird populations, verify community-sourced labels, etc.

The other motivation is more open-ended: one of curiosity and self-education that we hope will build towards better understanding the impacts of the "anthropony" (sounds produced by humans). As novice bird-listeners ourselves, we are amazed by the variability and information "encoded" in bird calls. Bird calls reveal regional dialects, a sense of humor, individual voices - and they are inevitably threatened by human activity. To our untrained ears, it's difficult to even distinguish calls from different species, let alone genders or pitch-shifts in reaction to noise pollution. Through the process visualizing and analyzing features of bird call audio, we hope we can build better understanding of bird calls and become better listeners.

In the next section, we will look at some of the previous work on intraspecies variation in bird calls.

# Related work on intraspecific variation in bird calls
## Gender identification using acoustic analysis in birds without external sexual dimorphism [[Link]] (https://avianres.biomedcentral.com/articles/10.1186/s40657-015-0033-y)
> Volodin, I.A., Volodina, E.V., Klenova, A.V. et al. Gender identification using acoustic analysis in birds without external sexual dimorphism. Avian Res 6, 20 (2015). https://doi.org/10.1186/s40657-015-0033-y

### Useful Terms
**Call** : Bird vocalization

**Song** : Courtship or territorial vocalization, typically from male songbirds in characteristic phrases

**Note** : Smallest unit of vocalization, an uninterrupted trace on the spectrogram

**Syllable** : Combination of notes that are separated by short intervals; syllables are separated by longer intervals

**Rhythmic structure** : The way notes, syllables, or calls repeat (count and rate of repetition)

**Fundamental frequency** : The call pitch, visible on a spectrogram usually as the lowest band in a stack of integer-multiple bands (harmonics)

**Harmonics** : Integer multiple frequency bands upon the fundamental frequency band.

**Frequency modulation** : Changes in fundamental frequency contour during a call

**Tonal call** : Call, containing the fundamental frequency and its related harmonics.

**Noisy call** : Call where the fundamental frequency and harmonics are indistinguishable or lacking from the inside. Looks like uniform noise on the spectrogram.

**Dominant frequency** : The frequency where the maximum energy of a call is concentrated.

### Background
- Determining the sex of adult birds using their calls is useful for monomorphic birds (no difference in appearance across sexes), is noninvasive, and may be done at a distance
- It is important to sex birds for wildlife management - breeding and census estimates
- Bird calls likely differ across sexes due to differences in morphology (e.g. size of beak, vocal organ, trachea), the method of producing a call, or in the type of calls/songs

### Contribution
- Researchers used spectrograms and power spectra
![spectrograms](https://media.springernature.com/lw685/springer-static/image/art%3A10.1186%2Fs40657-015-0033-y/MediaObjects/40657_2015_33_Fig1_HTML.gif?as=webp)
- They used simple visual inspection (without measuring acoustic variables) for certain species, or using Discriminate Function Analysis when acoustic variables overlapped more across sexes.
    - DFA accuracy > 70% was considered sufficient but low reliability for sexing by call (50% would be equivalent to random chance)
    - 91 - 100% : perfect reliability
- Relevant vocal features for differentiating sex were:
    - Average fundamental frequency
    - Maximum fundamental frequency
    - Biphonation (2 independent fundamental frequencies)
    - Duration of notes
    - Number of syllables in call
    - Amplitude modulation (wideband spectra)
    - Intervals between syallables

## Further Reading
- [2009 Classifying Bird Calls with Supervised Learning](https://eecs.oregonstate.edu/research/bioacoustics/briggs_icdm09.pdf) - useful background on audio features
- [How Birds Develop Song Dialects](https://sora.unm.edu/sites/default/files/journals/condor/v077n04/p0385-p0406.pdf) - regional dialects in birdsong
- [Do bird calls of the same species differ across countries?](https://www.researchgate.net/post/Does-bird-calls-of-same-species-from-one-country-differ-from-the-calls-from-other-country) - Researchgate forum, some more links here

## Regional dialects have been discovered among many bird species and the Yellowhammer is a great example [[Link]](https://bou.org.uk/yellowhammer-dialects/)

Birds have been observed to have regional dialects, as a result of ecological drivers, [human disturbances](https://academic.oup.com/beheco/article/30/6/1501/5526711?login=true), and cultural evolution.

A famous citizen science project took place in the Czech Republic starting in 2011 to study dialects of Yellowhammer birds (easily recognized). It revealed two main dialect groups and the border between, and brought up more questions like: How are the dialects maintained? What causes the dialect boundaries between neighboring habitats? How do dialects evolve?
![yellowhammer dialects](https://bou.org.uk/wp-content/uploads/2013/05/Pipek-fig2-530x296.png)

# Collecting Data
For our analysis, we will use audio files and metadata from xeno-canto.org. Xeno-canto (XC) is a website for collecting and sharing audio recordings of birds. 

## How does XC maintain data?
Recordings and identifications are sourced from the community (anyone can join), as are recording quality ratings and flags for wrong IDs.
### Audio quality
Recordings are rated by community members according to this criteria:
- A: Loud and Clear
- B: Clear, but bird a bit distant, or some interference with other sound sources
- C: Moderately clear, or quite some interference
- D: Faint recording, or much interference
- E: Barely audible

### Identification Accuracy
Recording IDs are submitted by users, and can be flagged as wrong by other community members. When an ID is flagged as wrong, it's removed from search results and the discussion is posted to the site forum and on the recording page.

## How do we get XC data?
XC has a [very simple API](https://www.xeno-canto.org/explore/api) that allows us to make RESTful queries, without even requiring an API key. A request url looks like this: `https://www.xeno-canto.org/api/2/recordings?query=bearded+bellbird+q:A&page=5`.

We can also test our request by going to `https://www.xeno-canto.org/explore?query=[our-query-here]`.

### Query parameter
A result is returned if the `Common name`, `Scientific name`, or `Scientific name of family` fields contain the query.

We can also combine tags to do advanced searches, using a `+` between each, for example:
- `cnt:"United States"` to get recordings from the US (quotes for multi-word countries)
- `loc:pennsylvania` to get recordings from PA
- `type:=female` to get bird calls ID'd as female (`=` forces a match)
- `len:10-15` to get recordings that are 10-15 seconds long
- `len_lt:120` to get recordings that are less than 2 minutes long
- `q_gt:C` for recordings greater than quality C

### Page parameter
The page parameter is only needed if the results don't fit on one page.

In [2]:
# make an example query here

## What does XC data look like?
We can see the JSON payload in the response includes a few self-explanatory fields, including an array of recording objects.
```
{
    "numRecordings": [the number of recordings found from the query],
    "numSpecies": [the number of species found from the query],
    "page": [current page number of results],
    "numPages": [total number of pages available],
    "recordings": [
        ...,
        Array of Recording objects
        ...
    ]
}
```
The recording objects likewise include a bunch of fields, nearly identical to the query parameters we could use to search the database. The fields we care most about are:
- **type** : a comma-separated list of the sound types of the recording (e.g. call, song, male, female)
- **file** : the url to the audio file
- **sono** : a dict with the urls to 4 sizes of sonograms (we want "full")
- **loc** : the location of the recording
- **lat** & **lng** : the latitude and longitude of the recording

In [3]:
from nbformat import read

# Default path for this notebook (can be run inside the notebook)
path = "/work/pracds_final/notebooks/0.0-mk-final-report.ipynb"
with open(path, 'r', encoding='utf-8') as f:
    nb = read(f, 4)

# Count the number of lines in markdown or heading cells
word_count = sum([
    len(cell['source'].replace('#', '').lstrip().split(' '))
    for cell in nb["cells"]
    if cell.cell_type in ["markdown", "heading"]
])

# Count number of lines in the notebook and subtract the number of
# lines in this cell
line_count = sum([
    # Filter out cells that are comments or are empty
    len(list(filter(
        lambda line: not (line.lstrip().startswith("#")),
        cell['source'].split("\n")
    )))
    for cell in nb["cells"]
    if cell.cell_type == "code"
]) - 27

print(f"Word Count: {word_count:,}")
print(f"Line Count: {line_count:,}")

Word Count: 1,267
Line Count: -4


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6b2ae77c-22c0-4e92-b8f6-ab962db6bbab' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>